In [1]:
import tkinter as tk
from PIL import Image, ImageTk
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, MultiPolygon
import pandas as pd
from country_list import available_languages, countries_for_language

In [2]:

# Function to update coordinates
def update_coordinates(geometry, scale_factor=1.0, translate_x=0.0, translate_y=0.0):
    if geometry.geom_type == 'Polygon':
        return Polygon([(x * scale_factor + translate_x, y * scale_factor + translate_y) for x, y in geometry.exterior.coords])
    elif geometry.geom_type == 'MultiPolygon':
        new_polygons = []
        for polygon in geometry.geoms:
            new_polygon = Polygon([(x * scale_factor + translate_x, y * scale_factor + translate_y) for x, y in polygon.exterior.coords])
            new_polygons.append(new_polygon)
        return MultiPolygon(new_polygons)
    else:
        return geometry
    

#world['geometry'] = world['geometry'].apply(update_coordinates, scale_factor=1.0, translate_x=180.0, translate_y=90)
#world['geometry'] = world['geometry'].apply(update_coordinates, scale_factor=2.5, translate_x=0, translate_y=0)


#world.total_bounds

In [5]:
### Processing the dataset and preparing data

# read world.csv as columns name_en and name_pl into already existing world dataframe
world_pd = pd.read_csv('world.csv', sep=' ')  # Adjusted separator to comma
world_pd['name_en'] = world_pd['name_en'].str.strip()
world_pd['name_pl'] = world_pd['name_pl'].str.strip()
world_pd['capital'] = world_pd['capital'].str.strip()

world = gpd.read_file(r'ne_110m_admin_0_countries.shp')
world = world[['NAME_LONG', 'POSTAL','CONTINENT','geometry']]
# rename NAME_LONG to name_eng, postal to iso
world.columns = ['name_en', 'iso', 'continent', 'geometry']
world

world["name_pl"] = world_pd["name_pl"]
world['capital'] = world_pd['capital']
#switch order of columns to name_en, name_pl, geometry and remove iso
world = world[['name_en', 'name_pl', 'capital', 'continent', 'geometry']]
world

world_exc = gpd.GeoDataFrame(columns=world.columns)
for i in range(world.shape[0]):
    geometry = world['geometry'].iloc[i]
    if geometry.geom_type == 'Polygon':
        new_row = world.iloc[i].copy()
        world_exc = world_exc.append(new_row, ignore_index=True)
    elif geometry.geom_type == 'MultiPolygon':
        for polygon in geometry.geoms:
            new_row = world.iloc[i].copy()
            new_row['geometry'] = polygon
            world_exc = world_exc.append(new_row, ignore_index=True)
world_exc

world = world_exc

# add a column providing unique id for every row
world['id'] = world.index
world

C:\Users\Radek\AppData\Local\Temp\ipykernel_23264\3154405195.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  world_exc = world_exc.append(new_row, ignore_index=True)
C:\Users\Radek\AppData\Local\Temp\ipykernel_23264\3154405195.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  world_exc = world_exc.append(new_row, ignore_index=True)
C:\Users\Radek\AppData\Local\Temp\ipykernel_23264\3154405195.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  world_exc = world_exc.append(new_row, ignore_index=True)
C:\Users\Radek\AppData\Local\Temp\ipykernel_23264\3154405195.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  world_exc = world

,name_en,name_pl,capital,continent,geometry,id
0,Fiji,Fidżi,Suva,Oceania,"POLYGON ((180 -16.06713, 180 -16.55522, 179.36...",0
1,Fiji,Fidżi,Suva,Oceania,"POLYGON ((178.12557 -17.50481, 178.3736 -17.33...",1
2,Fiji,Fidżi,Suva,Oceania,"POLYGON ((-179.79332 -16.02088, -179.91737 -16...",2
3,Tanzania,Tanzania,Dodoma,Africa,"POLYGON ((33.90371 -0.95, 34.07262 -1.05982, 3...",3
4,Western Sahara,Sahara Zachodnia,Apia,Africa,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",4
...,...,...,...,...,...,...
283,Serbia,Serbia,Belgrad,Europe,"POLYGON ((18.82982 45.90887, 18.82984 45.90888...",283
284,Montenegro,Czarnogóra,Podgorica,Europe,"POLYGON ((20.0707 42.58863, 19.80161 42.50009,...",284
285,Kosovo,Kosowo,Prisztina,Europe,"POLYGON ((20.59025 41.85541, 20.52295 42.21787...",285
286,Trinidad and Tobago,Trynidad i Tobago,Port-of-Spain,North America,"POLYGON ((-61.68 10.76, -61.105 10.89, -60.895...",286


In [6]:
world

,name_en,name_pl,capital,continent,geometry,id
0,Fiji,Fidżi,Suva,Oceania,"POLYGON ((180 -16.06713, 180 -16.55522, 179.36...",0
1,Fiji,Fidżi,Suva,Oceania,"POLYGON ((178.12557 -17.50481, 178.3736 -17.33...",1
2,Fiji,Fidżi,Suva,Oceania,"POLYGON ((-179.79332 -16.02088, -179.91737 -16...",2
3,Tanzania,Tanzania,Dodoma,Africa,"POLYGON ((33.90371 -0.95, 34.07262 -1.05982, 3...",3
4,Western Sahara,Sahara Zachodnia,Apia,Africa,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",4
...,...,...,...,...,...,...
283,Serbia,Serbia,Belgrad,Europe,"POLYGON ((18.82982 45.90887, 18.82984 45.90888...",283
284,Montenegro,Czarnogóra,Podgorica,Europe,"POLYGON ((20.0707 42.58863, 19.80161 42.50009,...",284
285,Kosovo,Kosowo,Prisztina,Europe,"POLYGON ((20.59025 41.85541, 20.52295 42.21787...",285
286,Trinidad and Tobago,Trynidad i Tobago,Port-of-Spain,North America,"POLYGON ((-61.68 10.76, -61.105 10.89, -60.895...",286


In [8]:
# display all rows where name_pl is Rosja
world[world['name_pl'] == 'Rosja']

# for all of those rows, change continent to Asia
world.loc[world['name_pl'] == 'Rosja', 'continent'] = 'Asia'


In [9]:
# EUROPE, DATASET PREPARATION
europe = world[world['continent'] == 'Europe'].copy()
europe = europe[europe['name_pl'] != 'Rosja'].copy()
europe

,name_en,name_pl,capital,continent,geometry,id
93,Norway,Norwegia,Oslo,Europe,"POLYGON ((15.14282 79.67431, 15.52255 80.01608...",93
94,Norway,Norwegia,Oslo,Europe,"POLYGON ((31.10104 69.5581, 29.39955 69.15692,...",94
95,Norway,Norwegia,Oslo,Europe,"POLYGON ((27.40751 80.05641, 25.92465 79.51783...",95
96,Norway,Norwegia,Oslo,Europe,"POLYGON ((24.72412 77.85385, 22.49032 77.44493...",96
118,France,Francja,Paryż,Europe,"POLYGON ((-51.6578 4.15623, -52.24934 3.24109,...",118
119,France,Francja,Paryż,Europe,"POLYGON ((6.18632 49.4638, 6.65823 49.20196, 8...",119
120,France,Francja,Paryż,Europe,"POLYGON ((8.74601 42.62812, 9.39 43.00998, 9.5...",120
191,Sweden,Szwecja,Sztokholm,Europe,"POLYGON ((11.02737 58.85615, 11.46827 59.43239...",191
192,Belarus,Białoruś,Mińsk,Europe,"POLYGON ((28.17671 56.16913, 29.22951 55.91834...",192
193,Ukraine,Ukraina,Kijów,Europe,"POLYGON ((31.78599 52.10168, 32.15944 52.06125...",193


In [10]:
# list all unique values in continent
world['continent'].unique()


array(['Oceania', 'Africa', 'North America', 'Asia', 'South America',
       'Europe', 'Seven seas (open ocean)', 'Antarctica'], dtype=object)

In [11]:
# ASIA, DATASET PREPARATION

asia = world[world['continent'] == 'Asia'].copy()
#europe = europe[europe['name_pl'] != 'Rosja'].copy()
asia

,name_en,name_pl,capital,continent,geometry,id
45,Kazakhstan,Kazachstan,Astana,Asia,"POLYGON ((87.35997 49.21498, 86.59878 48.54918...",45
46,Uzbekistan,Uzbekistan,Taszkent,Asia,"POLYGON ((55.96819 41.30864, 55.92892 44.99586...",46
51,Indonesia,Indonezja,Dżakarta,Asia,"POLYGON ((141.00021 -2.60015, 141.01706 -5.859...",51
52,Indonesia,Indonezja,Dżakarta,Asia,"POLYGON ((124.96868 -8.89279, 125.07002 -9.089...",52
53,Indonesia,Indonezja,Dżakarta,Asia,"POLYGON ((134.21013 -6.89524, 134.11278 -6.142...",53
...,...,...,...,...,...,...
259,Japan,Japonia,Tokio,Asia,"POLYGON ((132.37118 33.46364, 132.92437 34.060...",259
261,Yemen,Jemen,Sana,Asia,"POLYGON ((52.00001 19, 52.78218 17.34974, 53.1...",261
262,Saudi Arabia,Arabia Saudyjska,Rijad,Asia,"POLYGON ((34.95604 29.35655, 36.06894 29.19749...",262
271,Northern Cyprus,Cypr Północny,Nikozja Północna,Asia,"POLYGON ((32.73178 35.14003, 32.80247 35.1455,...",271


In [69]:
### AFRICA DATASET PREPARATION
africa = world[world['continent'] == 'Africa'].copy()
#europe = europe[europe['name_pl'] != 'Rosja'].copy()
# remove all rows where name_pl is "Lesotho"
africa = africa[africa['name_pl'] != 'Lesotho'].copy()
africa

,name_en,name_pl,capital,continent,geometry,id
3,Tanzania,Tanzania,Dodoma,Africa,"POLYGON ((33.90371 -0.95, 34.07262 -1.05982, 3...",3
4,Western Sahara,Sahara Zachodnia,Apia,Africa,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",4
68,Democratic Republic of the Congo,Demokratyczna Republika Konga,Kinszasa,Africa,"POLYGON ((29.34 -4.49998, 29.51999 -5.41998, 2...",68
69,Somalia,Somalia,Mogadiszu,Africa,"POLYGON ((41.58513 -1.68325, 40.993 -0.85829, ...",69
70,Kenya,Kenia,Nairobi,Africa,"POLYGON ((39.20222 -4.67677, 37.7669 -3.67712,...",70
71,Sudan,Sudan,Chartum,Africa,"POLYGON ((24.56737 8.22919, 23.80581 8.66632, ...",71
72,Chad,Czad,Ndżamena,Africa,"POLYGON ((23.83766 19.58047, 23.88689 15.61084...",72
100,South Africa,Republika Południowej Afryki,Pretoria,Africa,"POLYGON ((16.34498 -28.57671, 16.82402 -28.082...",100
125,Zimbabwe,Zimbabwe,Harare,Africa,"POLYGON ((31.19141 -22.25151, 30.65987 -22.151...",125
126,Botswana,Botswana,Gaborone,Africa,"POLYGON ((29.43219 -22.09131, 28.01724 -22.827...",126


In [101]:
central_america = world[world['continent'] == 'North America'].copy()
# remove all rows where name_pl is "Grenlandia", Stany Zjednoczone, Kanada
central_america = central_america[central_america['name_pl'] != 'Grenlandia'].copy()
central_america = central_america[central_america['name_pl'] != 'Stany Zjednoczone'].copy()
central_america = central_america[central_america['name_pl'] != 'Kanada'].copy()
central_america

,name_en,name_pl,capital,continent,geometry,id
73,Haiti,Haiti,Port-au-Prince,North America,"POLYGON ((-71.71236 19.71446, -71.62487 19.169...",73
74,Dominican Republic,Dominikana,Santo Domingo,North America,"POLYGON ((-71.7083 18.045, -71.68774 18.31666,...",74
89,Bahamas,Bahamy,Nassau,North America,"POLYGON ((-78.98 26.79, -78.51 26.87, -77.85 2...",89
90,Bahamas,Bahamy,Nassau,North America,"POLYGON ((-77.79 27.04, -77 26.59, -77.17255 2...",90
91,Bahamas,Bahamy,Nassau,North America,"POLYGON ((-78.19087 25.2103, -77.89 25.17, -77...",91
102,Mexico,Meksyk,Meksyk,North America,"POLYGON ((-117.12776 32.53534, -115.99135 32.6...",102
108,Panama,Panama,Panama,North America,"POLYGON ((-77.35336 8.6705, -77.47472 8.52429,...",108
109,Costa Rica,Kostaryka,San Jose,North America,"POLYGON ((-82.5462 9.56613, -82.93289 9.47681,...",109
110,Nicaragua,Nikaragua,Managua,North America,"POLYGON ((-83.65561 10.93876, -83.89505 10.726...",110
111,Honduras,Honduras,Tegucigalpa,North America,"POLYGON ((-83.14722 14.99583, -83.48999 15.016...",111


In [110]:
south_america = world[world['continent'] == 'South America'].copy()
south_america

,name_en,name_pl,capital,continent,geometry,id
64,Argentina,Argentyna,Buenos Aires,South America,"POLYGON ((-68.63401 -52.63637, -68.25 -53.1, -...",64
65,Argentina,Argentyna,Buenos Aires,South America,"POLYGON ((-57.62513 -30.21629, -57.87494 -31.0...",65
66,Chile,Chile,Santiago,South America,"POLYGON ((-68.63401 -52.63637, -68.63335 -54.8...",66
67,Chile,Chile,Santiago,South America,"POLYGON ((-69.59042 -17.58001, -69.10025 -18.2...",67
92,Falkland Islands / Malvinas,Falklandy,Stanley,South America,"POLYGON ((-61.2 -51.85, -60 -51.25, -59.15 -51...",92
103,Uruguay,Urugwaj,Montevideo,South America,"POLYGON ((-57.62513 -30.21629, -56.97603 -30.1...",103
104,Brazil,Brazylia,Brasilia,South America,"POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...",104
105,Bolivia,Boliwia,Sucre,South America,"POLYGON ((-69.52968 -10.95173, -68.78616 -11.0...",105
106,Peru,Peru,Lima,South America,"POLYGON ((-69.89364 -4.29819, -70.79477 -4.251...",106
107,Colombia,Kolumbia,Bogota,South America,"POLYGON ((-66.87633 1.25336, -67.06505 1.13011...",107


In [130]:
australia_ocenia = world[world['continent'] == 'Oceania'].copy()
australia_ocenia

,name_en,name_pl,capital,continent,geometry,id
0,Fiji,Fidżi,Suva,Oceania,"POLYGON ((180 -16.06713, 180 -16.55522, 179.36...",0
1,Fiji,Fidżi,Suva,Oceania,"POLYGON ((178.12557 -17.50481, 178.3736 -17.33...",1
2,Fiji,Fidżi,Suva,Oceania,"POLYGON ((-179.79332 -16.02088, -179.91737 -16...",2
47,Papua New Guinea,Papua-Nowa Gwinea,Port Moresby,Oceania,"POLYGON ((141.00021 -2.60015, 142.73525 -3.289...",47
48,Papua New Guinea,Papua-Nowa Gwinea,Port Moresby,Oceania,"POLYGON ((152.64002 -3.65998, 153.01999 -3.980...",48
49,Papua New Guinea,Papua-Nowa Gwinea,Port Moresby,Oceania,"POLYGON ((151.30139 -5.84073, 150.75445 -6.083...",49
50,Papua New Guinea,Papua-Nowa Gwinea,Port Moresby,Oceania,"POLYGON ((154.75999 -5.33998, 155.06292 -5.566...",50
168,Vanuatu,Vanuatu,Port Vila,Oceania,"POLYGON ((167.2168 -15.89185, 167.84488 -16.46...",168
169,Vanuatu,Vanuatu,Port Vila,Oceania,"POLYGON ((166.79316 -15.66881, 166.64986 -15.3...",169
217,New Caledonia,Nowa Kaledonia,Numea,Oceania,"POLYGON ((165.77999 -21.08, 166.59999 -21.7000...",217


In [13]:
####### EUROPA, Ostateczna wersja ready to go

import tkinter as tk
from shapely.geometry import Polygon, MultiPolygon

def create_map_from_geodataframe(geodf):
    # Create the main window
    # create a copy of dataset, so it cannot be modified globally
    geodf = geodf.copy()

    # Adjust coordinates
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=1.0, translate_x=25, translate_y=-30)
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=16.5, translate_x=0, translate_y=0)

    root = tk.Tk()
    root.title("Europe Map")
    canvas_width, canvas_height = 1100, 950
    root.geometry(f"{canvas_width}x{canvas_height}+400+10")
    canvas = tk.Canvas(root, width=canvas_width, height=canvas_height)
    canvas.pack()

    # Function to handle mouse clicks
    def on_canvas_click(event):
        x, y = event.x, event.y
        for idx, row in geodf.iterrows():
            geometry = row['geometry']
            if isinstance(geometry, Polygon):
                coords = list(geometry.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                if is_point_in_polygon(x, y, inverted_coords):
                    highlight_elements(row['name_pl'], "yellow")
                    show_popup(row['name_pl'], row['capital'])
                    return
            elif isinstance(geometry, MultiPolygon):
                for polygon in geometry.geoms:
                    coords = list(polygon.exterior.coords)
                    inverted_coords = [(x, canvas_height - y) for x, y in coords]
                    if is_point_in_polygon(x, y, inverted_coords):
                        highlight_elements(row['name_pl'], "yellow")
                        show_popup(row['name_pl'], row['capital'])
                        return

    # Function to check if a point is inside a polygon
    def is_point_in_polygon(x, y, polygon):
        n = len(polygon)
        inside = False
        p1x, p1y = polygon[0]
        for i in range(n + 1):
            p2x, p2y = polygon[i % n]
            if y > min(p1y, p2y):
                if y <= max(p1y, p2y):
                    if x <= max(p1x, p2x):
                        if p1y != p2y:
                            xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                        if p1x == p2x or x <= xinters:
                            inside = not inside
            p1x, p1y = p2x, p2y
        return inside

    # Function to show the popup window
    def show_popup(name_pl, capital):
        popup = tk.Toplevel(root)
        popup.title("Country Information")
        popup.geometry("200x200")  # Set the size of the popup window

        tk.Label(popup, text="Enter Country Name:").pack(pady=5)
        country_entry = tk.Entry(popup)
        country_entry.pack(pady=5)

        tk.Label(popup, text="Enter Capital:").pack(pady=5)
        capital_entry = tk.Entry(popup)
        capital_entry.pack(pady=5)

        def check_answer():
            entered_country = country_entry.get()
            entered_capital = capital_entry.get()
            if entered_country == name_pl and entered_capital == capital:
                highlight_elements(name_pl, "green")
            else:
                highlight_elements(name_pl, "red")
                incorrect_label = tk.Label(root, text=f"Ten kraj to {name_pl}, a jego stolica to {capital}", fg="red")
                incorrect_label.pack()
                root.after(3000, incorrect_label.destroy)
            popup.destroy()

        tk.Button(popup, text="Submit", command=check_answer).pack(pady=10)

    # Function to highlight elements with the same name_pl
    def highlight_elements(name_pl, color):
        for idx, row in geodf.iterrows():
            if row['name_pl'] == name_pl:
                canvas.itemconfig(row['id'], fill=color)

    # Draw polygons on the canvas
    for idx, row in geodf.iterrows():
        geometry = row['geometry']
        if isinstance(geometry, Polygon):
            coords = list(geometry.exterior.coords)
            inverted_coords = [(x, canvas_height - y) for x, y in coords]
            item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
            geodf.at[idx, 'id'] = item_id
        elif isinstance(geometry, MultiPolygon):
            for polygon in geometry.geoms:
                coords = list(polygon.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
                geodf.at[idx, 'id'] = item_id

    canvas.bind("<Button-1>", on_canvas_click)

    # Run the main loop
    root.mainloop()

# Call the function with the europe GeoDataFrame
create_map_from_geodataframe(europe)

In [46]:
####### AZJA, Ostateczna wersja ready to go

import tkinter as tk
from shapely.geometry import Polygon, MultiPolygon

def create_map_from_geodataframe(geodf):
    # Create the main window
    # create a copy of dataset, so it cannot be modified globally
    geodf = geodf.copy()

    # Adjust coordinates
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=1.0, translate_x=-15, translate_y=10)
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=10.5, translate_x=0, translate_y=0)

    root = tk.Tk()
    root.title("Asia Map")
    canvas_width, canvas_height = 1800, 950
    canvas_height_adjusted = canvas_height + 50
    root.geometry(f"{canvas_width}x{canvas_height_adjusted}+100+10")
    canvas = tk.Canvas(root, width=canvas_width, height=canvas_height)
    canvas.pack()

    # Function to handle mouse clicks
    def on_canvas_click(event):
        x, y = event.x, event.y
        for idx, row in geodf.iterrows():
            geometry = row['geometry']
            if isinstance(geometry, Polygon):
                coords = list(geometry.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                if is_point_in_polygon(x, y, inverted_coords):
                    highlight_elements(row['name_pl'], "yellow")
                    show_popup(row['name_pl'], row['capital'])
                    return
            elif isinstance(geometry, MultiPolygon):
                for polygon in geometry.geoms:
                    coords = list(polygon.exterior.coords)
                    inverted_coords = [(x, canvas_height - y) for x, y in coords]
                    if is_point_in_polygon(x, y, inverted_coords):
                        highlight_elements(row['name_pl'], "yellow")
                        show_popup(row['name_pl'], row['capital'])
                        return

    # Function to check if a point is inside a polygon
    def is_point_in_polygon(x, y, polygon):
        n = len(polygon)
        inside = False
        p1x, p1y = polygon[0]
        for i in range(n + 1):
            p2x, p2y = polygon[i % n]
            if y > min(p1y, p2y):
                if y <= max(p1y, p2y):
                    if x <= max(p1x, p2x):
                        if p1y != p2y:
                            xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                        if p1x == p2x or x <= xinters:
                            inside = not inside
            p1x, p1y = p2x, p2y
        return inside

    # Function to show the popup window
    def show_popup(name_pl, capital):
        popup = tk.Toplevel(root)
        popup.title("Country Information")
        popup.geometry("200x200")  # Set the size of the popup window

        tk.Label(popup, text="Enter Country Name:").pack(pady=5)
        country_entry = tk.Entry(popup)
        country_entry.pack(pady=5)

        tk.Label(popup, text="Enter Capital:").pack(pady=5)
        capital_entry = tk.Entry(popup)
        capital_entry.pack(pady=5)

        def check_answer():
            entered_country = country_entry.get()
            entered_capital = capital_entry.get()
            if entered_country == name_pl and entered_capital == capital:
                highlight_elements(name_pl, "green")
            else:
                highlight_elements(name_pl, "red")
                incorrect_label = tk.Label(root, text=f"Ten kraj to {name_pl}, a jego stolica to {capital}", fg="red")
                incorrect_label.pack()
                root.after(3000, incorrect_label.destroy)
            popup.destroy()

        tk.Button(popup, text="Submit", command=check_answer).pack(pady=10)

    # Function to highlight elements with the same name_pl
    def highlight_elements(name_pl, color):
        for idx, row in geodf.iterrows():
            if row['name_pl'] == name_pl:
                canvas.itemconfig(row['id'], fill=color)

    # Draw polygons on the canvas
    for idx, row in geodf.iterrows():
        geometry = row['geometry']
        if isinstance(geometry, Polygon):
            coords = list(geometry.exterior.coords)
            inverted_coords = [(x, canvas_height - y) for x, y in coords]
            item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
            geodf.at[idx, 'id'] = item_id
        elif isinstance(geometry, MultiPolygon):
            for polygon in geometry.geoms:
                coords = list(polygon.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
                geodf.at[idx, 'id'] = item_id

    canvas.bind("<Button-1>", on_canvas_click)

    # Run the main loop
    root.mainloop()

# Call the function with the europe GeoDataFrame
create_map_from_geodataframe(asia)

In [70]:
####### AFRYKA, Ostateczna wersja ready to go

import tkinter as tk
from shapely.geometry import Polygon, MultiPolygon

def create_map_from_geodataframe(geodf):
    # Create the main window
    # create a copy of dataset, so it cannot be modified globally
    geodf = geodf.copy()

    # Adjust coordinates
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=1.0, translate_x=30, translate_y=35)
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=12.5, translate_x=0, translate_y=0)

    root = tk.Tk()
    root.title("Africa Map")
    canvas_width, canvas_height = 1100, 950
    canvas_height_adjusted = canvas_height + 50
    root.geometry(f"{canvas_width}x{canvas_height_adjusted}+300+10")
    canvas = tk.Canvas(root, width=canvas_width, height=canvas_height)
    canvas.pack()

    # Function to handle mouse clicks
    def on_canvas_click(event):
        x, y = event.x, event.y
        for idx, row in geodf.iterrows():
            geometry = row['geometry']
            if isinstance(geometry, Polygon):
                coords = list(geometry.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                if is_point_in_polygon(x, y, inverted_coords):
                    highlight_elements(row['name_pl'], "yellow")
                    show_popup(row['name_pl'], row['capital'])
                    return
            elif isinstance(geometry, MultiPolygon):
                for polygon in geometry.geoms:
                    coords = list(polygon.exterior.coords)
                    inverted_coords = [(x, canvas_height - y) for x, y in coords]
                    if is_point_in_polygon(x, y, inverted_coords):
                        highlight_elements(row['name_pl'], "yellow")
                        show_popup(row['name_pl'], row['capital'])
                        return

    # Function to check if a point is inside a polygon
    def is_point_in_polygon(x, y, polygon):
        n = len(polygon)
        inside = False
        p1x, p1y = polygon[0]
        for i in range(n + 1):
            p2x, p2y = polygon[i % n]
            if y > min(p1y, p2y):
                if y <= max(p1y, p2y):
                    if x <= max(p1x, p2x):
                        if p1y != p2y:
                            xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                        if p1x == p2x or x <= xinters:
                            inside = not inside
            p1x, p1y = p2x, p2y
        return inside

    # Function to show the popup window
    def show_popup(name_pl, capital):
        popup = tk.Toplevel(root)
        popup.title("Country Information")
        popup.geometry("200x200")  # Set the size of the popup window

        tk.Label(popup, text="Enter Country Name:").pack(pady=5)
        country_entry = tk.Entry(popup)
        country_entry.pack(pady=5)

        tk.Label(popup, text="Enter Capital:").pack(pady=5)
        capital_entry = tk.Entry(popup)
        capital_entry.pack(pady=5)

        def check_answer():
            entered_country = country_entry.get()
            entered_capital = capital_entry.get()
            if entered_country == name_pl and entered_capital == capital:
                highlight_elements(name_pl, "green")
            else:
                highlight_elements(name_pl, "red")
                incorrect_label = tk.Label(root, text=f"Ten kraj to {name_pl}, a jego stolica to {capital}", fg="red")
                incorrect_label.pack()
                root.after(3000, incorrect_label.destroy)
            popup.destroy()

        tk.Button(popup, text="Submit", command=check_answer).pack(pady=10)

    # Function to highlight elements with the same name_pl
    def highlight_elements(name_pl, color):
        for idx, row in geodf.iterrows():
            if row['name_pl'] == name_pl:
                canvas.itemconfig(row['id'], fill=color)

    # Draw polygons on the canvas
    for idx, row in geodf.iterrows():
        geometry = row['geometry']
        if isinstance(geometry, Polygon):
            coords = list(geometry.exterior.coords)
            inverted_coords = [(x, canvas_height - y) for x, y in coords]
            item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
            geodf.at[idx, 'id'] = item_id
        elif isinstance(geometry, MultiPolygon):
            for polygon in geometry.geoms:
                coords = list(polygon.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
                geodf.at[idx, 'id'] = item_id

    canvas.bind("<Button-1>", on_canvas_click)

    # Run the main loop
    root.mainloop()

# Call the function with the europe GeoDataFrame
create_map_from_geodataframe(africa)

In [109]:
####### AMERYKA ŚRODKOWA, Ostateczna wersja ready to go

import tkinter as tk
from shapely.geometry import Polygon, MultiPolygon

def create_map_from_geodataframe(geodf):
    # Create the main window
    # create a copy of dataset, so it cannot be modified globally
    geodf = geodf.copy()

    # Adjust coordinates
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=1.0, translate_x=120, translate_y=5)
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=18, translate_x=0, translate_y=0)

    root = tk.Tk()
    root.title("Central America Map")
    canvas_width, canvas_height = 1100, 950
    canvas_height_adjusted = canvas_height + 50
    root.geometry(f"{canvas_width}x{canvas_height_adjusted}+300+10")
    canvas = tk.Canvas(root, width=canvas_width, height=canvas_height)
    canvas.pack()

    # Function to handle mouse clicks
    def on_canvas_click(event):
        x, y = event.x, event.y
        for idx, row in geodf.iterrows():
            geometry = row['geometry']
            if isinstance(geometry, Polygon):
                coords = list(geometry.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                if is_point_in_polygon(x, y, inverted_coords):
                    highlight_elements(row['name_pl'], "yellow")
                    show_popup(row['name_pl'], row['capital'])
                    return
            elif isinstance(geometry, MultiPolygon):
                for polygon in geometry.geoms:
                    coords = list(polygon.exterior.coords)
                    inverted_coords = [(x, canvas_height - y) for x, y in coords]
                    if is_point_in_polygon(x, y, inverted_coords):
                        highlight_elements(row['name_pl'], "yellow")
                        show_popup(row['name_pl'], row['capital'])
                        return

    # Function to check if a point is inside a polygon
    def is_point_in_polygon(x, y, polygon):
        n = len(polygon)
        inside = False
        p1x, p1y = polygon[0]
        for i in range(n + 1):
            p2x, p2y = polygon[i % n]
            if y > min(p1y, p2y):
                if y <= max(p1y, p2y):
                    if x <= max(p1x, p2x):
                        if p1y != p2y:
                            xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                        if p1x == p2x or x <= xinters:
                            inside = not inside
            p1x, p1y = p2x, p2y
        return inside

    # Function to show the popup window
    def show_popup(name_pl, capital):
        popup = tk.Toplevel(root)
        popup.title("Country Information")
        popup.geometry("200x200")  # Set the size of the popup window

        tk.Label(popup, text="Enter Country Name:").pack(pady=5)
        country_entry = tk.Entry(popup)
        country_entry.pack(pady=5)

        tk.Label(popup, text="Enter Capital:").pack(pady=5)
        capital_entry = tk.Entry(popup)
        capital_entry.pack(pady=5)

        def check_answer():
            entered_country = country_entry.get()
            entered_capital = capital_entry.get()
            if entered_country == name_pl and entered_capital == capital:
                highlight_elements(name_pl, "green")
            else:
                highlight_elements(name_pl, "red")
                incorrect_label = tk.Label(root, text=f"Ten kraj to {name_pl}, a jego stolica to {capital}", fg="red")
                incorrect_label.pack()
                root.after(3000, incorrect_label.destroy)
            popup.destroy()

        tk.Button(popup, text="Submit", command=check_answer).pack(pady=10)

    # Function to highlight elements with the same name_pl
    def highlight_elements(name_pl, color):
        for idx, row in geodf.iterrows():
            if row['name_pl'] == name_pl:
                canvas.itemconfig(row['id'], fill=color)

    # Draw polygons on the canvas
    for idx, row in geodf.iterrows():
        geometry = row['geometry']
        if isinstance(geometry, Polygon):
            coords = list(geometry.exterior.coords)
            inverted_coords = [(x, canvas_height - y) for x, y in coords]
            item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
            geodf.at[idx, 'id'] = item_id
        elif isinstance(geometry, MultiPolygon):
            for polygon in geometry.geoms:
                coords = list(polygon.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
                geodf.at[idx, 'id'] = item_id

    canvas.bind("<Button-1>", on_canvas_click)

    # Run the main loop
    root.mainloop()

# Call the function with the europe GeoDataFrame
create_map_from_geodataframe(central_america)

In [125]:
####### AMERYKA Południowa, Ostateczna wersja ready to go

import tkinter as tk
from shapely.geometry import Polygon, MultiPolygon

def create_map_from_geodataframe(geodf):
    # Create the main window
    # create a copy of dataset, so it cannot be modified globally
    geodf = geodf.copy()

    # Adjust coordinates
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=1.0, translate_x=100, translate_y=56)
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=13.8, translate_x=0, translate_y=0)

    root = tk.Tk()
    root.title("South America Map")
    canvas_width, canvas_height = 1100, 950
    canvas_height_adjusted = canvas_height + 50
    root.geometry(f"{canvas_width}x{canvas_height_adjusted}+300+10")
    canvas = tk.Canvas(root, width=canvas_width, height=canvas_height)
    canvas.pack()

    # Function to handle mouse clicks
    def on_canvas_click(event):
        x, y = event.x, event.y
        for idx, row in geodf.iterrows():
            geometry = row['geometry']
            if isinstance(geometry, Polygon):
                coords = list(geometry.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                if is_point_in_polygon(x, y, inverted_coords):
                    highlight_elements(row['name_pl'], "yellow")
                    show_popup(row['name_pl'], row['capital'])
                    return
            elif isinstance(geometry, MultiPolygon):
                for polygon in geometry.geoms:
                    coords = list(polygon.exterior.coords)
                    inverted_coords = [(x, canvas_height - y) for x, y in coords]
                    if is_point_in_polygon(x, y, inverted_coords):
                        highlight_elements(row['name_pl'], "yellow")
                        show_popup(row['name_pl'], row['capital'])
                        return

    # Function to check if a point is inside a polygon
    def is_point_in_polygon(x, y, polygon):
        n = len(polygon)
        inside = False
        p1x, p1y = polygon[0]
        for i in range(n + 1):
            p2x, p2y = polygon[i % n]
            if y > min(p1y, p2y):
                if y <= max(p1y, p2y):
                    if x <= max(p1x, p2x):
                        if p1y != p2y:
                            xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                        if p1x == p2x or x <= xinters:
                            inside = not inside
            p1x, p1y = p2x, p2y
        return inside

    # Function to show the popup window
    def show_popup(name_pl, capital):
        popup = tk.Toplevel(root)
        popup.title("Country Information")
        popup.geometry("200x200")  # Set the size of the popup window

        tk.Label(popup, text="Enter Country Name:").pack(pady=5)
        country_entry = tk.Entry(popup)
        country_entry.pack(pady=5)

        tk.Label(popup, text="Enter Capital:").pack(pady=5)
        capital_entry = tk.Entry(popup)
        capital_entry.pack(pady=5)

        def check_answer():
            entered_country = country_entry.get()
            entered_capital = capital_entry.get()
            if entered_country == name_pl and entered_capital == capital:
                highlight_elements(name_pl, "green")
            else:
                highlight_elements(name_pl, "red")
                incorrect_label = tk.Label(root, text=f"Ten kraj to {name_pl}, a jego stolica to {capital}", fg="red")
                incorrect_label.pack()
                root.after(3000, incorrect_label.destroy)
            popup.destroy()

        tk.Button(popup, text="Submit", command=check_answer).pack(pady=10)

    # Function to highlight elements with the same name_pl
    def highlight_elements(name_pl, color):
        for idx, row in geodf.iterrows():
            if row['name_pl'] == name_pl:
                canvas.itemconfig(row['id'], fill=color)

    # Draw polygons on the canvas
    for idx, row in geodf.iterrows():
        geometry = row['geometry']
        if isinstance(geometry, Polygon):
            coords = list(geometry.exterior.coords)
            inverted_coords = [(x, canvas_height - y) for x, y in coords]
            item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
            geodf.at[idx, 'id'] = item_id
        elif isinstance(geometry, MultiPolygon):
            for polygon in geometry.geoms:
                coords = list(polygon.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
                geodf.at[idx, 'id'] = item_id

    canvas.bind("<Button-1>", on_canvas_click)

    # Run the main loop
    root.mainloop()

# Call the function with the europe GeoDataFrame
create_map_from_geodataframe(south_america)

In [168]:
####### AUSTRALIA I OCEANIA, Ostateczna wersja ready to go

import tkinter as tk
from shapely.geometry import Polygon, MultiPolygon

def create_map_from_geodataframe(geodf):
    # Create the main window
    # create a copy of dataset, so it cannot be modified globally
    geodf = geodf.copy()

    # Adjust coordinates
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=1.0, translate_x=-100, translate_y=60)
    geodf['geometry'] = geodf['geometry'].apply(update_coordinates, scale_factor=12, translate_x=0, translate_y=0)

    root = tk.Tk()
    root.title("South America Map")
    canvas_width, canvas_height = 1100, 950
    canvas_height_adjusted = canvas_height + 50
    root.geometry(f"{canvas_width}x{canvas_height_adjusted}+300+10")
    canvas = tk.Canvas(root, width=canvas_width, height=canvas_height)
    canvas.pack()

    # Function to handle mouse clicks
    def on_canvas_click(event):
        x, y = event.x, event.y
        for idx, row in geodf.iterrows():
            geometry = row['geometry']
            if isinstance(geometry, Polygon):
                coords = list(geometry.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                if is_point_in_polygon(x, y, inverted_coords):
                    highlight_elements(row['name_pl'], "yellow")
                    show_popup(row['name_pl'], row['capital'])
                    return
            elif isinstance(geometry, MultiPolygon):
                for polygon in geometry.geoms:
                    coords = list(polygon.exterior.coords)
                    inverted_coords = [(x, canvas_height - y) for x, y in coords]
                    if is_point_in_polygon(x, y, inverted_coords):
                        highlight_elements(row['name_pl'], "yellow")
                        show_popup(row['name_pl'], row['capital'])
                        return

    # Function to check if a point is inside a polygon
    def is_point_in_polygon(x, y, polygon):
        n = len(polygon)
        inside = False
        p1x, p1y = polygon[0]
        for i in range(n + 1):
            p2x, p2y = polygon[i % n]
            if y > min(p1y, p2y):
                if y <= max(p1y, p2y):
                    if x <= max(p1x, p2x):
                        if p1y != p2y:
                            xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                        if p1x == p2x or x <= xinters:
                            inside = not inside
            p1x, p1y = p2x, p2y
        return inside

    # Function to show the popup window
    def show_popup(name_pl, capital):
        popup = tk.Toplevel(root)
        popup.title("Country Information")
        popup.geometry("200x200")  # Set the size of the popup window

        tk.Label(popup, text="Enter Country Name:").pack(pady=5)
        country_entry = tk.Entry(popup)
        country_entry.pack(pady=5)

        tk.Label(popup, text="Enter Capital:").pack(pady=5)
        capital_entry = tk.Entry(popup)
        capital_entry.pack(pady=5)

        def check_answer():
            entered_country = country_entry.get()
            entered_capital = capital_entry.get()
            if entered_country == name_pl and entered_capital == capital:
                highlight_elements(name_pl, "green")
            else:
                highlight_elements(name_pl, "red")
                incorrect_label = tk.Label(root, text=f"Ten kraj to {name_pl}, a jego stolica to {capital}", fg="red")
                incorrect_label.pack()
                root.after(3000, incorrect_label.destroy)
            popup.destroy()

        tk.Button(popup, text="Submit", command=check_answer).pack(pady=10)

    # Function to highlight elements with the same name_pl
    def highlight_elements(name_pl, color):
        for idx, row in geodf.iterrows():
            if row['name_pl'] == name_pl:
                canvas.itemconfig(row['id'], fill=color)

    # Draw polygons on the canvas
    for idx, row in geodf.iterrows():
        geometry = row['geometry']
        if isinstance(geometry, Polygon):
            coords = list(geometry.exterior.coords)
            inverted_coords = [(x, canvas_height - y) for x, y in coords]
            item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
            geodf.at[idx, 'id'] = item_id
        elif isinstance(geometry, MultiPolygon):
            for polygon in geometry.geoms:
                coords = list(polygon.exterior.coords)
                inverted_coords = [(x, canvas_height - y) for x, y in coords]
                item_id = canvas.create_polygon(inverted_coords, outline='black', fill='blue', width=2)
                geodf.at[idx, 'id'] = item_id

    canvas.bind("<Button-1>", on_canvas_click)

    # Run the main loop
    root.mainloop()

# Call the function with the europe GeoDataFrame
create_map_from_geodataframe(australia_ocenia)